# Inversion of Control

- Main theme : Objects don't create dependent objects in their methods or constructors. Instead, the container manages them and injects them as needed
- Spring maintains the lifecycle of classes (or Beans) and handles injection
- Construction and management of object is no longer developer's responsibility
- Objects are injected as dependencies in other classes during runtime or start-up time
- Allows developer to focus on business logic based on contracts (interface)
- Developers often need to build only abstraction and not the whole implementation details
- Spring serves singleton objects from the bean factory by default
- Bean Factory : IoC Container.
- Application Context : wraps around Bean Factory. Developers interact with it to indirectly work with Bean Factory
- References : Bean Factory maintains references to the Spring Beans that either developer or Spring itself configures
- Analysis of Construction Order : after initialization of beans, an order of operation is created based on the dependencies and the beans are then instantiated according to proper order by the Bean Factory
- Recommended injections
    - Constructor injection
    - Setter injection
    - NOT FIELD injection
- IOC Workflow
    - Application starts
    - IOC Container scans for classes
    - Constructs objects according to order in the bean factory
    - Injects the dependencies according to the order
    - NOTE : It will fail if there is circular dependency

# Service Abstraction

- Encapsulation Layer : to hide lower level code
- Consuming third-party APIs and protect the code
- Simplify implementation and reduces complexity (Imrpoves understandablity of the code)
- Isolate implementation, Swappable code (no code change necessary)
- Workflow of building Service Abstraction:
    - Define class or interface
    - Create API
    - Inject dependency
    - Annotate or Configure
    - Code the implementation (Code and test together)

# Creating Service

### `Model`

- In the base package > a new package `Service` > A new package `Model` 
- Should contain the fields from the entities

```
@Getter
@Setter
@ToString
public class RoomReservation {
    private Long reservationId;
    private String roomNumber;
    private String guestName;
    private Date reservationDate;
}
```

### Interface of the Service : `RoomReservationService`

```
import java.util.List;

public interface RoomReservationService {
    List<RoomReservation> getReservationsByDate(Date reservationDate);
}

```

### Implementation Class : `RoomReservationServiceImpl`

```

@Service
public class RoomReservationServiceImpl implements RoomReservationService {

    @Autowired
    private ReservationRepository reservationRepository;

    @Override
    public List<RoomReservation> getReservationsByDate(Date reservationDate) {
    List<RoomReservation> roomReservations = new ArrayList<>();
    List<Reservation> reservations = reservationRepository.findAllByReservationDate(reservationDate);
    
    for (Reservation reservation : reservations) {
        Long roomId = reservation.getRoomId();
        Long guestId = reservation.getGuestId();
        
        // Fetch Room object using roomId
        Room room = roomRepository.findById(roomId).orElse(null);
        
        // Fetch Guest object using guestId
        Guest guest = guestRepository.findById(guestId).orElse(null);
        
        if (room != null && guest != null) {
            RoomReservation roomReservation = new RoomReservation();
            roomReservation.setReservationId(reservation.getReservationId());
            roomReservation.setRoomNumber(room.getRoomNumber());
            roomReservation.setGuestName(guest.getGuestName());
            roomReservation.setReservationDate(reservation.getReservationDate());
            
            roomReservations.add(roomReservation);
        }
    }
    
    return roomReservations;
}
}
```

### Test with command line

```

@Component
public class CLRunner implements CommandLineRunner {

    @Autowired
    private final RoomReservationService roomReservationService; // the interface

    @Override
    public void run(String... args) throws Exception {
        // Specify the desired reservation date
        SimpleDateFormat dateFormat = new SimpleDateFormat("yyyy-MM-dd");
        Date reservationDate = dateFormat.parse("2022-02-14");
        // test with the specified date.... You are using the custom model 
        List<RoomReservation> roomReservations = roomReservationService.getReservationsByDate(reservationDate);

        // Display the room reservations in the console with getters
        for (RoomReservation roomReservation : roomReservations) {
            System.out.println("Reservation Date: " + dateFormat.format(roomReservation.getReservationDate()));
            // other getters
        }
    }
}

```